# DeepER Classic VS VinSIM + DeepER (Fine Tuning)

## Step 0: Caricamento dati, preprocessing e strutture ausiliarie

In [ ]:
from DeepER import init_embeddings_index, init_embeddings_model, init_DeepER_model, train_model_ER, replace_last_layer, model_statistics
from experimental_similarity import mono_vector, mono_vector_fast, cosine_similarity_vector, distance_similarity_vector
from csv2dataset import splitting_dataSet, csv_2_datasetALTERNATE, csvTable2datasetRANDOM
from generate_similarity_vector import generate_similarity_vector
from keras.models import load_model
from keras.layers import Dense
from plotly import graph_objs as go
import plotly.offline as pyo
from random import shuffle
import utilist as uls

In [ ]:
# Imposta manualmente a True per caricare da disco tutti i modelli salvati. 
# Imposta manualmente a False per ri-eseguire tutti gli addestramenti.
LOAD_FROM_DISK = False
# Il nome con cui saranno etichettati i files prodotti
DATASET_NAME = # Esempio: 'WA'

In [ ]:
# Caricamento strutture dati e modelli ausiliari.
embeddings_index = init_embeddings_index('glove.6B.50d.txt')
emb_dim = len(embeddings_index['cat']) # :3
embeddings_model, tokenizer = init_embeddings_model(embeddings_index)

In [ ]:
# Caricamento dati e split iniziale.
if LOAD_FROM_DISK:
    
    # Carica dataset salvato su disco.
    data = uls.load_list(f'dataset_{DATASET_NAME}')

else:
    
    GROUND_TRUTH_FILE = # Esempio: 'matches_walmart_amazon.csv'
    # Necessario inserire le tabelle nell'ordine corrispondente alle coppie della ground truth.
    TABLE1_FILE = # Esempio: 'walmart.csv'
    TABLE2_FILE = # Esempio: 'amazon.csv'

    # Coppie di attributi considerati allineati.
    att_indexes =  [(1, 1), (2, 2), (3, 3), (4, 4)] # Esempio: [(5, 9), (4, 5), (3, 3), (14, 4), (6, 11)]

    # Similarity callbacks
    simf = lambda a, b: cosine_similarity_vector(a, b, embeddings_index)
    #simf = lambda a, b: mono_vector_fast(a, b)
    #simf = lambda a, b: mono_vector(a, b)

    # Crea il dataset.
    data = csv_2_datasetALTERNATE(GROUND_TRUTH_FILE, TABLE1_FILE, TABLE2_FILE, att_indexes, simf)
    
    # Salva dataset su disco.
    uls.save_list(data, f'dataset_{DATASET_NAME}')

    
# Dataset per DeepER classico: [(tupla1, tupla2, label), ...].
deeper_data = list(map(lambda q: (q[0], q[1], q[3]), data))


# Taglia attributi se troppo lunghi
# Alcuni dataset hanno attributi con descrizioni molto lunghe.
# Questo filtro limita il numero di caratteri di un attributo a 1000.
def shrink_data(data):
    
    def cut_string(s):
        if len(s) >= 700:
            return s[:700]
        else:
            return s
    
    temp = []
    for t1, t2, lb in data:
        t1 = list(map(cut_string, t1))
        t2 = list(map(cut_string, t2))
        temp.append((t1, t2, lb))
        
    return temp

deeper_data = shrink_data(deeper_data)


# Split in training set e test set.
def split_training_test(data, SPLIT_FACTOR = 0.8):     
    bound = int(len(data) * SPLIT_FACTOR)
    train = data[:bound]
    test = data[bound:]
    
    return train, test


# Tutti i successivi addestramenti partiranno dal 100% di deeper_train (80% di tutti i dati).
# Le tuple in deeper_test non verranno mai usate per addestrare ma solo per testare i modelli.
deeper_train, deeper_test = split_training_test(deeper_data)

## Step 1: Addestramento standard

In [ ]:
# InPut: Percentuale di dati considerata per l'addestramento. 
# OutPut: DeepER addestrato sul taglio scelto.
def get_DeepER(perc):
   
    sub_data = splitting_dataSet(perc, deeper_train)    
    
    if LOAD_FROM_DISK:
        
        # Carica da disco.
        print(f'Loading DeepER_best_model_{int(perc*100)}_{DATASET_NAME}.h5', end='', flush=True)
        deeper_model = load_model(f'DeepER_best_model_{int(perc*100)}_{DATASET_NAME}.h5')
        print('  ->  Done')        
                
    else:
        
        # Inizializza il modello.
        deeper_model = init_DeepER_model(emb_dim)
        deeper_model.summary()
        # Avvio addestramento.
        deeper_model = train_model_ER(sub_data, 
                                      deeper_model, 
                                      embeddings_model, 
                                      tokenizer, 
                                      pretraining=False, 
                                      end=f'_{int(perc*100)}_{DATASET_NAME}')
        
    return deeper_model

In [ ]:
# Avvio addestramenti o carica da disco.
deeper_model_5 = get_DeepER(0.05)
deeper_model_10 = get_DeepER(0.1)
deeper_model_25 = get_DeepER(0.25)
deeper_model_50 = get_DeepER(0.5)
deeper_model_75 = get_DeepER(0.75)
deeper_model_100 = get_DeepER(1)

### Calcolo F-Measure dopo addestramento standard

In [ ]:
# Misurazione dell'f-measure sullo stesso test set con i diversi modelli.
fm_model_standard = [model_statistics(deeper_test, deeper_model_100, embeddings_model, tokenizer),
                     model_statistics(deeper_test, deeper_model_75, embeddings_model, tokenizer),
                     model_statistics(deeper_test, deeper_model_50, embeddings_model, tokenizer),
                     model_statistics(deeper_test, deeper_model_25, embeddings_model, tokenizer),
                     model_statistics(deeper_test, deeper_model_10, embeddings_model, tokenizer),
                     model_statistics(deeper_test, deeper_model_5, embeddings_model, tokenizer)]

print(fm_model_standard)

### Visualizzazione F-Measure: primi risultati

In [ ]:
# Attiva modalità notebook per mostrare i grafici correttamente.
pyo.init_notebook_mode()

splits = ['100% split', '75% split', '50% split', '25% split', '10% split', '5% split']
total_tup = len(deeper_train)
tuplecount = [total_tup, 
              int(total_tup*0.75), 
              int(total_tup*0.5), 
              int(total_tup*0.25), 
              int(total_tup*0.1), 
              int(total_tup*0.05)]

# Aggiungi descrizione al numero
tuplecount = list(map(lambda x: f'{x} coppie di tuple', tuplecount))

fig = go.Figure(data=[go.Bar(name='DeepER', x=splits, y=fm_model_standard, hovertext=tuplecount)])

#fig.show()

# Plotta il grafico e salvalo come features_standard.html (verrà integrato nell'html).
pyo.iplot(fig, filename='fmeasures-standard')

##### Al passaggio del mouse il grafico mostra:
- Il numero di coppie di tuple utilizzate per l'addestramento; 
- La percentuale di split (Quantità di tuple utilizzate per addestrare il modello);
- Il valore di F-Measure (media armonica tra precision e recall);

## Step 2: Addestramento con Pre-Train

### Pre-Addestramento modello VinSim

In [ ]:
# Caricamento o addestramento del modello per la similarità.
if LOAD_FROM_DISK:    
    
    vinsim_model = load_model(f'VinSim_best_model_{DATASET_NAME}.h5')
    
else:
    
    # Dataset per VinSim.
    vinsim_data = []
    
    # Porzione di tuple in match da includere nell'addestramento di VinSim.
    TP_FACTOR = 0.2
    
    # Preleva solo quelle in match con il relativo sim vector.
    for i in range(len(data)):
        if data[i][3] == 1:
            r = data[i]
            vinsim_data.append((r[0], r[1], r[2]))
            
    # Taglio della porzione desiderata.
    bound = int(len(vinsim_data) * TP_FACTOR)
    vinsim_data = vinsim_data[:bound]
    
    # Generazione di tuple random.
    random_tuples = csvTable2datasetRANDOM(TABLE1_FILE, TABLE2_FILE, len(data)*2, att_indexes, simf)
    
    # Concatenazione.
    vinsim_data += random_tuples
    
    # Shuffle.
    shuffle(vinsim_data)
    
    # Filtro.
    vinsim_data = shrink_data(vinsim_data)
        
    # Inizializza un nuovo modello.
    vinsim_model = init_DeepER_model(emb_dim)

    # Sostituisci ultimo layer e ricompila per l'addestramento.
    output_neurons = len(vinsim_data[0][2]) # Parametrico rispetto alla dimensione del vettore di similarità.
    vinsim_model = replace_last_layer(vinsim_model, Dense(output_neurons, activation='sigmoid', name='VinSim'))    
    vinsim_model.compile(optimizer='adam', loss='mean_squared_error')
    vinsim_model.summary()

    # Avvia l'addestramento. 
    train_model_ER(vinsim_data, 
                   vinsim_model, 
                   embeddings_model, 
                   tokenizer, 
                   pretraining=True, 
                   metric='val_loss', 
                   end=f'_{DATASET_NAME}')

### Addestramento VinSim + DeepER

In [ ]:
# Input: Percentuale di dati considerata per l'addestramento.
# Output: DeepER addestrato con preaddestramento VinSim.
def get_PreTrained(perc):
   
    sub_data = splitting_dataSet(perc, deeper_train)       
    
    if LOAD_FROM_DISK:
        
        # Carica da disco.      
        print(f'Loading DeepER_best_model_{int(perc*100)}_pre_{DATASET_NAME}.h5', end='', flush=True)
        deeper_model_pre = load_model(f'DeepER_best_model_{int(perc*100)}_pre_{DATASET_NAME}.h5')
        print('  ->  Done') 
        
    else:
        
        # Utilizza il modello addestrato sulla similarità per l'addestramento (transfer learning).
        deeper_model_pre = load_model(f'VinSim_best_model_{DATASET_NAME}.h5')
        deeper_model_pre = replace_last_layer(deeper_model_pre, Dense(2, activation='softmax', name='Classification'))
        deeper_model_pre.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
        deeper_model_pre.summary()
        deeper_model_pre = train_model_ER(sub_data, 
                                          deeper_model_pre, 
                                          embeddings_model, 
                                          tokenizer, 
                                          pretraining=False, 
                                          end=f'_{int(perc*100)}_pre_{DATASET_NAME}')     
    
    return deeper_model_pre

In [ ]:
# Avvio addestramenti o carica da disco.
deeper_model_5_pre = get_PreTrained(0.05)
deeper_model_10_pre = get_PreTrained(0.1)
deeper_model_25_pre = get_PreTrained(0.25)
deeper_model_50_pre = get_PreTrained(0.5)
deeper_model_75_pre = get_PreTrained(0.75)
deeper_model_100_pre = get_PreTrained(1)

### F-Measure per VinSim + DeepER

In [ ]:
# Misurazione dell'f-measure sullo stesso test set con i diversi modelli.
fm_model_pre_trained = [model_statistics(deeper_test, deeper_model_100_pre, embeddings_model, tokenizer),
                        model_statistics(deeper_test, deeper_model_75_pre, embeddings_model, tokenizer),
                        model_statistics(deeper_test, deeper_model_50_pre, embeddings_model, tokenizer),
                        model_statistics(deeper_test, deeper_model_25_pre, embeddings_model, tokenizer),
                        model_statistics(deeper_test, deeper_model_10_pre, embeddings_model, tokenizer),
                        model_statistics(deeper_test, deeper_model_5_pre, embeddings_model, tokenizer)]

print(fm_model_pre_trained)

## Visualizzazione F-Measure: comparazione finale

In [ ]:
fig2 = go.Figure(data=[go.Bar(name='DeepER standard', x=splits, y=fm_model_standard),
                       go.Bar(name='VinSim + DeepER', x=splits, y=fm_model_pre_trained)])

# Modalità di visualizzazione con colonne raggruppate.
fig2.update_layout(barmode='group')
#fig.show()

# Plotta il grafico e salvalo come features_comparison.html (verrà integrato nell'html).
pyo.iplot(fig2, filename='fmeasures-comparison')